# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [263]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [264]:
def movie_title_url_actor():
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html)
    
    # 1) 영화제목 긁어오기
    tit = soup.select('dt[class="tit"] a', limit = 5)

    titles = []
    for i in range(len(tit)):
        titles.append(tit[i].get_text())
        
    # 2) 주연배우 긁어오기
    actor_list = soup.select('span[class="link_txt"]')
    actors = [] # 5개영화 주연배우 모두 들어갈 리스트
    #actor_list를 뽑았을때 주연배우가 3을 주기로 나타나기 때문에 3*i+2 수열 식을 이용하여 배우명 뽑아내기
    for i in range(5): #영화5개에 대해
        actor = []  #각 영화별로 주연배우 리스트
        if len(actor_list[3*i + 2].find_all('a')) < 3: #1917예외처리를 위한 코드(주연배우가 3명이하인 경우)
            for x in actor_list[3*i + 2].find_all('a'):
                actor.append(x.get_text())
            actors.append(actor)
        else:
            for j in range(3): #3명까지만 actor에 넣어주기
                actor.append(actor_list[3*i + 2].find_all('a')[j].get_text())
            actors.append(actor)
    
    # 3) 제목, 배우 합치기
    #tit_act = dict(zip(titles, actors))
    df = pd.DataFrame(index = range(0,5),columns = ['영화제목','주연배우'])
    df['영화제목'] = titles
    df['주연배우'] = actors
    df['주연배우'] = df['주연배우'].str.join(',') #리스트형태로 들어오기 때문에 리스트 기호 없애주기
     
    return df

### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [265]:
def insert_grades():
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html)
    
    # 1) 영화 별 url 주소 뽑아내기
    tit = soup.select('dt[class="tit"] a', limit = 5)
    tit_url = [] #리스트로 생성
    for i in range(len(tit)):
        tit_url.append("https://movie.naver.com" + tit[i].attrs['href'])

    
    # 2) 평점 가져오기
    data = movie_title_url_actor()
    # df에 새로운 평점 column들 생성(어차피 밑에서 차례로 값 채울거라 일단 모든 값들 0으로 지정해둠)
    data['관람객 평점'] = 0
    data['기자,평론가 평점'] = 0
    data['네티즌 평점'] = 0
    
    for i in range(5): #각 영화별로 접속하자
        url = tit_url[i]
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html)

        score = soup.select('div[class="star_score"] em')

        review = []
        for j in range(8): #첫순서부터 8개 값 뽑아내면 4개씩 관람객평점, 기자평론가 평점이 나온다
            review.append(score[j].get_text())

        data['관람객 평점'][i] = review[0]+review[1]+review[2]+review[3]
        data['기자,평론가 평점'][i] = review[4]+review[5]+review[6]+review[7]
        data['네티즌 평점'][i] = score[-7].get_text() #네티즌평점은 저 밑에 혼자 있길래, 영화마다 위치가 동일하길래 인덱싱으로 불러옴
        
    return data

    


### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [266]:
def insert_reviews():
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html)
    
    # 1) 영화제목, 영화별 url주소 긁어오기
    tit = soup.select('dt[class="tit"] a', limit = 5)
    # 영화제목
    titles = []
    for t in range(len(tit)):
        titles.append(tit[t].get_text())
    # url 주소 뽑아내기
    tit_url = [] #리스트로 생성
    for i in range(len(tit)):
        tit_url.append("https://movie.naver.com" + tit[i].attrs['href'])
        
    #일단 tit_url에서 영화별 코드만 뽑아오자
    news=[]
    for j in tit_url:
        new = re.sub('[^0-9\s]', '', j)
        news.append(new)
    #영화별로 2개 페이지씩 리뷰 url 생성(한페이지당 리뷰 10개)
    review_url = []
    for w in news:
        review_url.append("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=" + w + "&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1")
        review_url.append("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=" + w + "&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2")
    
    #새로운 데이터프레임 생성
    df1 = pd.DataFrame(index = range(0,100),columns = ['영화제목','닉네임(아이디)','평점','리뷰'])
    for u in range(5):
        df1['영화제목'][20*u:20*u+20] = titles[u]
    
    
    for e in range(len(review_url)):
        url = review_url[e]
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html)

        #닉네임(아이디)
        names = []
        review_name = soup.select('div[class="score_reple"] a')
        # 이렇게 a 섹션만 불러왔을때 규칙성이 있는 와중에 욕설이 섞인 리뷰는 한번 더 표시가 되기 때문에 그냥 돌리면 오류가 난다. 그래서
        for k in range(len(review_name)): #밑에 조건문을 걸었다.
            if review_name[k].find('span') == None: # 아이디가 들어있는 span 섹션에 혹시 아무것도 없다면
                pass #넘어가라
            else: #있다면 names에 추가해라
                names.append(review_name[k].find('span').get_text())
        df1['닉네임(아이디)'][10*e:10*e+10] = names #이 url에 해당하는 영화의 행에 차례로 다 들어가라
        
        #평점 #닉네임과 마찬가지
        review_score = soup.select('div[class="star_score"] em')
        for p in range(len(review_score)):
            review_score[p] = review_score[p].get_text()
        df1['평점'][10*e:10*e+10] = review_score
        
        #리뷰
        sooo_review_text = soup.select('div[class="score_reple"] p')
        # 리뷰는 get_text 걸어줘도 남아있는 \n,\t 이런 개행문자들을 전부 제거해주어야해서 함수를 만들었다
        def space_del(text): 
            text1 = re.sub('&nbsp; | &nbsp;| \n|\t|\r','',text)
            text2 = re.sub('\n\n|\n','', text1)
            return text2
        review_text = []
        for m in sooo_review_text:
            review_text.append(space_del(m.get_text()))
        df1['리뷰'][10*e:10*e+10] = review_text
    
    return df1

### 4. 저장하기

In [267]:
def save():
    data = insert_grades()
    review_data = insert_reviews()
    
    # csv로 저장했더니 #데이터타입 다 깨지면서 난리가 남...
    #data.to_csv("naver_movie_top5.csv")
    #review_data.to_csv("naver_movie_top5_review.csv")
    
    # 엑셀로 저장
    data.to_excel("naver_movie_top5.xlsx")
    review_data.to_excel("naver_movie_top5_review.xlsx")
    
    # pickle로 저장
    # 피클은 파이썬의 모든 객체를 파일로 저장할 수 있는 방법으로 sklearn으로 모델학습후 저장할때 많이 사용한다
    data.to_pickle("naver_movie_top5.pkl")
    review_data.to_pickle("naver_movie_top5_review.pkl")

### 5. 크롤링하기

In [268]:
data = movie_title_url_actor()

C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [269]:
data = insert_grades()
data

C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a 

,영화제목,주연배우,관람객 평점,"기자,평론가 평점",네티즌 평점
0,지푸라기라도 잡고 싶은 짐승들,"전도연,정우성,배성우",8.39,6.71,6.88
1,정직한 후보,"라미란,김무열,나문희",8.61,5.38,7.71
2,1917,"조지 맥케이,딘-찰스 채프먼",9.43,7.67,8.99
3,작은 아씨들,"시얼샤 로넌,엠마 왓슨,플로렌스 퓨",9.21,8.00,8.88
4,클로젯,"하정우,김남길,허율",8.39,5.50,6.86


In [270]:
#영화리뷰는 따로 저장
review_data = insert_reviews()
review_data

C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


,영화제목,닉네임(아이디),평점,리뷰
0,지푸라기라도 잡고 싶은 짐승들,어쩌라고(dpfk****),1,아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽...
1,지푸라기라도 잡고 싶은 짐승들,bohemian(mabu****),10,"난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 ..."
2,지푸라기라도 잡고 싶은 짐승들,최정규(cjg4****),10,전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
3,지푸라기라도 잡고 싶은 짐승들,달다(fxko****),10,8명의 배우가 모두 주인공 같은 느낌.
4,지푸라기라도 잡고 싶은 짐승들,써니(tlag****),9,개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ
...,...,...,...,...
95,클로젯,지애(maln****),1,관람객최근에 본 것 중에 제일 최악..
96,클로젯,삡(jiny****),10,김남길 눈빛이 다했다..재밌어요
97,클로젯,maimai(enma****),10,진짜 심장 쫄깃하고 무서움 ㄷㄷㄷ 이건 극장에서 봐야함
98,클로젯,난꽃이다(play****),10,무서운거 진짜 못보는대 혼자 봤거든요 걱정하시는분들 그걸 감안하고라도 보시는거 추천...


In [271]:
#결측치 있나 저장하기 전에 한번 확인해보자
review_data.isnull().sum()

영화제목        0
닉네임(아이디)    0
평점          0
리뷰          0
dtype: int64

결측치 없이 아주 잘 채워졌다......!!!!!!!!

In [272]:
#저장합시다~
save()

C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a 